# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving informaton out of natural language (could be seqeuences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## Get helper functions

In [2]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster).

See the original source here: https://www.kaggle.com/c/nlp-getting-started

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip


# Unzip data
unzip_data("nlp_getting_started.zip")

!rm "nlp_getting_started.zip"

--2025-01-16 15:22:47--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.66.251, 142.250.71.91, 142.250.76.123, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.66.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   751KB/s    in 0.8s    

2025-01-16 15:22:48 (751 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

In [3]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [5]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
# How many examples of each class?
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [7]:
# How many total samples?
len(train_df_shuffled), len(test_df)

(7613, 3263)

In [8]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 1 (real disaster)
Text:
NWS has issued a Severe Thunderstorm Warning for Oklahoma-OK until 10:30 PM

---

Target: 1 (real disaster)
Text:
78 passengers evacuated safely after Green Line train derails. http://t.co/KzBSOhtwB4

---

Target: 1 (real disaster)
Text:
Some of the aftershocks can be just as big as the initial earthquake.~ http://t.co/HKbPqdncBa

---

Target: 1 (real disaster)
Text:
Critters climate and two plague deaths in Colorado http://t.co/DXkt2Shuj2

---

Target: 0 (not real disaster)
Text:
I peeped you frontin' I was in the jeepåÊsunk in the seat tinted with heat beats bumpin'.

---



## Split data into training and validation sets

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # Use 10% of training data for validation split
                                                                            random_state=42)

In [11]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [12]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:

* Tokenziation - direct mapping of token (a token could be a word or a character) to number

* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [13]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int", # How to map token to numbers
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False)

2025-01-18 15:17:49.183595: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-01-18 15:17:49.183635: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-01-18 15:17:49.183641: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-01-18 15:17:49.183826: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-18 15:17:49.183844: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [16]:
# Setup text vectorization variables
max_vocab_length = 10000 # Max number of words to have in our vocabulary
max_length = 15 

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \n\nVectorized version: ")
text_vectorizer([random_sentence])

Original text:
 FAAN orders evacuation of abandoned aircraft at MMA http://t.co/GsOMtDPmoJ 

Vectorized version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1679, 1268,  245,    6, 1441,  661,   17, 2041,    1,    0,    0,
           0,    0,    0,    0]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # Get all the unique words in ourr training data
top_5_words = words_in_vocab[:5] # Get the most common words
bottom_5_words = words_in_vocab[-5:] # Get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating an Embedding using an Embedding Layer

In [21]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # Set input shape
                             output_dim=128, # Output shape
                             input_length=max_length # How long each input
                             )
embedding

/opt/anaconda3/envs/tensorflow_venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding, built=False>

In [22]:
random_sentence = random.choice(train_sentences)
print(f"Original text: \n {random_sentence}\n\n Embedding version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: 
 Apperception bridgework blight: XxhJeSC http://t.co/yBhBArajXp

 Embedding version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01069287, -0.03761255,  0.03297651, ..., -0.03587136,
         -0.01539639,  0.04193733],
        [-0.01069287, -0.03761255,  0.03297651, ..., -0.03587136,
         -0.01539639,  0.04193733],
        [-0.03196084, -0.01132579,  0.0145025 , ..., -0.00382328,
         -0.049203  , -0.04203482],
        ...,
        [ 0.00349072, -0.01429874, -0.00248802, ...,  0.04438804,
          0.04522033,  0.04167346],
        [ 0.00349072, -0.01429874, -0.00248802, ...,  0.04438804,
          0.04522033,  0.04167346],
        [ 0.00349072, -0.01429874, -0.00248802, ...,  0.04438804,
          0.04522033,  0.04167346]]], dtype=float32)>

In [23]:
# Check out single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.01069287, -0.03761255,  0.03297651,  0.03082451, -0.04317229,
        -0.0238081 ,  0.02592093,  0.02454287, -0.0033071 ,  0.04213269,
        -0.03740187,  0.02870083, -0.02896014,  0.01861005,  0.00239651,
        -0.02299426,  0.0433053 ,  0.00809874,  0.00366622,  0.04880575,
         0.04121455,  0.02440207, -0.01805814, -0.00254507,  0.03014094,
         0.02873758,  0.01421979,  0.04711782,  0.02728354,  0.00975465,
         0.02371733, -0.00357356,  0.04187169, -0.04043826, -0.02486467,
        -0.04337096, -0.03385811, -0.02979499,  0.01861178, -0.01452316,
         0.00086266,  0.02577848, -0.01395207, -0.02766954,  0.00076239,
        -0.02958851,  0.00892867,  0.03252287,  0.03437177,  0.01063813,
        -0.0234515 , -0.01290692,  0.03573387, -0.04015163,  0.00524013,
        -0.035379  ,  0.02036453,  0.04782548, -0.0290113 , -0.0117103 ,
        -0.03758453, -0.01334336, -0.0312613 , -0.04215275, -0.04278226,
  

## Modelling a text dataset (Running a series of experiments)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # Convert words to numbers using tfidf
    ("clf", MultinomialNB())  # Model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score * 100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [26]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

Let's make a function to evaluate our modelling experiment predictions using:

* Accuracy
* Precision
* Recall
* F1-score

In [27]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculate model accuracy, precision, recall and f1-score of a binary classification model. 
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100

    # Calculate model precision, recall and f1-score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {
        "accuracy": model_accuracy,
        "precision": model_precision,
        "recall": model_recall,
        "f1": model_f1
    }
    return model_results

In [28]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels, y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [29]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1, ), dtype=tf.string) # inputs are 1-dimensional string
x = text_vectorizer(inputs) # Turn the input text into numbers
x = embedding(x) # Create an embedding of the numberized inputs
# x = layers.GlobalAveragePooling1D()(x) # Condense the featuer vector for each token to one vector
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # Binary output so using sigmoid activation function
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [30]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [32]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5


2025-01-18 15:18:31.066297: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.5973 - loss: 0.6678 - val_accuracy: 0.7651 - val_loss: 0.5662
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8258 - loss: 0.4902 - val_accuracy: 0.7927 - val_loss: 0.4717
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8746 - loss: 0.3468 - val_accuracy: 0.7966 - val_loss: 0.4504
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9109 - loss: 0.2609 - val_accuracy: 0.7979 - val_loss: 0.4562
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9317 - loss: 0.2057 - val_accuracy: 0.7953 - val_loss: 0.4697


In [33]:
# Check the results 
model_1.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7818 - loss: 0.5211


[0.46972307562828064, 0.7952755689620972]

In [34]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


(762, 1)

In [35]:
model_1_pred_probs[0]

array([0.41381702], dtype=float32)

In [36]:
model_1_pred_probs[:10]

array([[0.41381702],
       [0.8673262 ],
       [0.99377835],
       [0.04833693],
       [0.26922384],
       [0.9603052 ],
       [0.95491433],
       [0.98861676],
       [0.93361807],
       [0.08647409]], dtype=float32)

In [37]:
# Convert model prediction probabilities to label format
model_1_pred_probs = tf.squeeze(tf.round(model_1_pred_probs))
model_1_pred_probs[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [38]:
import numpy as np

In [39]:
# Calculate model_1 results
model_1_results = calculate_results(y_true=val_labels, 
                                    y_pred=np.array(model_1_pred_probs))
model_1_results

{'accuracy': 79.52755905511812,
 'precision': 0.8019883406690347,
 'recall': 0.7952755905511811,
 'f1': 0.791960345124057}

In [40]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Visualizing learned embeddings

In [41]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [42]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [43]:
# Get the weight matrix of embedding layer
# (the weights are the numerical patterns between the text in the training dataset that the model has learned)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape) # Same size as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)


In [44]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premise of recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

### Model 2: LSTM

LSTM = Long short term memory (One of the most popular LSTM cells)

Our structure of an RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)
```
`return_sequences` - Boolean. Whether to return the last output in the output sequence, or the full sequence. Default: False

`inputs`: A 3D tensor with shape [batch, timesteps, feature]

In [45]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(64, return_sequences=True)(x) # When you're stacking RNN cells together use return_sequence True
# print(f"After LSTM cell with return_sequences=True: {x.shape}")
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [46]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [47]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [48]:
# Fit the model
model_2_history =  model_2.fit(train_sentences,
                               train_labels,
                               epochs=5,
                               validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - accuracy: 0.7745 - loss: 0.4675 - val_accuracy: 0.7940 - val_loss: 0.4775
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9125 - loss: 0.2323 - val_accuracy: 0.7598 - val_loss: 0.4982
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9399 - loss: 0.1698 - val_accuracy: 0.7848 - val_loss: 0.5252
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9557 - loss: 0.1256 - val_accuracy: 0.7822 - val_loss: 0.7040
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9642 - loss: 0.0932 - val_accuracy: 0.7559 - val_loss: 1.0043


In [49]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


array([[0.02700111],
       [0.77958924],
       [0.9998425 ],
       [0.02537954],
       [0.00265942],
       [0.99994874],
       [0.72973865],
       [0.99994934],
       [0.9999517 ],
       [0.41176823]], dtype=float32)

In [50]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [51]:
# Calculate model_2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=np.array(model_2_preds))
model_2_results

{'accuracy': 75.59055118110236,
 'precision': 0.7562023016353725,
 'recall': 0.7559055118110236,
 'f1': 0.75433369124314}

In [52]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 3: GRU

GRU = Gated recurrent unit (one of the most popular and useful recurrent layer types)

In [53]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1, ), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.GRU(64, return_sequences=True)(x) # if you want to stack recurrent layers on top of each other return_sequence must be True
x = layers.GRU(64, activation="tanh")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [54]:
# Get a summary of model_3
model_3.summary()

Model: "model_3_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,317,313 (5.03 MB)

 Trainable params: 1,317,313 (5.03 MB)

 Non-trainable params: 0 (0.00 B)

In [55]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [56]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - accuracy: 0.8458 - loss: 0.3246 - val_accuracy: 0.7861 - val_loss: 0.6578
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9684 - loss: 0.0919 - val_accuracy: 0.7795 - val_loss: 0.8578
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.9735 - loss: 0.0732 - val_accuracy: 0.7769 - val_loss: 0.7844
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9762 - loss: 0.0617 - val_accuracy: 0.7638 - val_loss: 1.1181
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9770 - loss: 0.0507 - val_accuracy: 0.7664 - val_loss: 1.2825


In [57]:
# Make some predictions with our GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


array([[7.5636263e-04],
       [6.3754493e-01],
       [9.9975127e-01],
       [2.7495382e-02],
       [1.5659202e-04],
       [9.9983883e-01],
       [9.3732321e-01],
       [9.9990189e-01],
       [9.9987876e-01],
       [2.1799581e-01]], dtype=float32)

In [58]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [59]:
# Calculate model_3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=np.array(model_3_preds))
model_3_results

{'accuracy': 76.64041994750657,
 'precision': 0.7735537522095453,
 'recall': 0.7664041994750657,
 'f1': 0.7619063300073503}

In [60]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 4: Bidirectional RNN model

Normal RNN's go from left to right however, a bidirectional RNN goes from right to left as well as left to right.

In [61]:
# Build a bidirectional RNN in TensorFlow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1, ), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [62]:
model_4.summary()

Model: "model_4_bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,378,945 (5.26 MB)

 Trainable params: 1,378,945 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

In [63]:
# Compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit model
history_model_4 = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 22s 86ms/step - accuracy: 0.8981 - loss: 0.2558 - val_accuracy: 0.7677 - val_loss: 0.7498
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 17s 79ms/step - accuracy: 0.9709 - loss: 0.0643 - val_accuracy: 0.7743 - val_loss: 1.0914
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 17s 78ms/step - accuracy: 0.9788 - loss: 0.0475 - val_accuracy: 0.7625 - val_loss: 1.2771
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 17s 78ms/step - accuracy: 0.9821 - loss: 0.0437 - val_accuracy: 0.7756 - val_loss: 1.0470
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 17s 78ms/step - accuracy: 0.9809 - loss: 0.0419 - val_accuracy: 0.7546 - val_loss: 1.3423


In [64]:
# Make predictions with our bidirectional model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10], model_4_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step


(array([[1.0011049e-04],
        [6.7524076e-01],
        [9.9994802e-01],
        [2.8791741e-01],
        [2.1041527e-05],
        [9.9974746e-01],
        [8.3455909e-03],
        [9.9999583e-01],
        [9.9999344e-01],
        [3.3062857e-01]], dtype=float32),
 (762, 1))

In [65]:
# Convert pred probs to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float32)>

In [66]:
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=np.array(model_4_preds))
model_4_results

{'accuracy': 75.45931758530183,
 'precision': 0.7607708781565117,
 'recall': 0.7545931758530183,
 'f1': 0.7499652364822783}

## Model 5: Conv1D

We've seen before how convolutional neural networks can be used for images but they can also be used for text.

Previously we've used the layer Conv2D (which is great for images with (height, width)).

But if we want to use convolutional layers for sequences (e.g. text) we need to use Conv1D:

In [74]:
# Test out our embedding layer, Conv1D layer and max pooling
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # Turn target sequence into embedding
conv_1d = layers.Conv1D(filters=64, # How many hidden units in our Conv1D layer
                        kernel_size=5, # It looks at 5 words at a time 
                        strides=1, # default - Slide over one word at a time. If strides is 2 it will hop two words at a time.
                        activation="relu", # max(0, z) - If the value is negative it gets converted to zero, if positive stays the same
                        padding="same") # default = "valid", output is smaller than the input shape, "same" means output is same shape as input
conv_1d_output = conv_1d(embedding_test) # Pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D() # Condense it into a singular feature vector
max_pool_output = max_pool(conv_1d_output) # equivalent to "get the most important feature" or "get the feature with the highest value"

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 64]), TensorShape([1, 64]))